In [17]:
from langchain.chat_models import ChatOpenAI
from langchain.chains.conversation.memory import ConversationBufferWindowMemory

In [19]:
# Set up the turbo LLM
turbo_llm = ChatOpenAI(
    model_name='gpt-3.5-turbo',
    temperature=0
)

In [26]:
from langchain.tools import GoogleSerperRun, GoogleSerperResults
from langchain.agents import Tool
from langchain.tools import BaseTool

search = GoogleSerperResults()
# defining a single tool
tools = [
    Tool(
        name = "search",
        func=search.run,
        description="useful for when you need to answer questions about current events. You should ask targeted questions"
    )
]

In [27]:
def meaning_of_life(input=""):
    return 'The meaning of life is 42 if rounded but is actually 42.17658'

life_tool = Tool(
    name='Meaning of Life',
    func= meaning_of_life,
    description="Useful for when you need to answer questions about the meaning of life. input should be MOL "
)

In [28]:
import random

def random_num(input=""):
    return random.randint(0,5)

random_tool = Tool(
    name='Random number',
    func= random_num,
    description="Useful for when you need to get a random number. input should be 'random'"
)

In [29]:
from langchain.agents import initialize_agent, agent_types

tools = [search, random_tool, life_tool]

# conversational agent memory
memory = ConversationBufferWindowMemory(
    memory_key='chat_history',
    k=3,
    return_messages=True
)

# create our agent
conversational_agent = initialize_agent(
    # agent='chat-conversational-react-description',
    agent=agent_types.AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION,
    tools=tools,
    llm=turbo_llm,
    verbose=True,
    max_iterations=3,
    early_stopping_method='generate',
    memory=memory
)

In [54]:
conversational_agent('What is the meaning of life?')



> Entering new AgentExecutor chain...
{
    "action": "Meaning of Life",
    "action_input": "MOL"
}
Observation: The meaning of life is 42 if rounded but is actually 42.17658
Thought:{
    "action": "Final Answer",
    "action_input": "The meaning of life is 42 if rounded but is actually 42.17658"
}

> Finished chain.


{'input': 'What is the meaning of life?',
 'chat_history': [HumanMessage(content='What is the meaning of life?', additional_kwargs={}, example=False),
  AIMessage(content='The meaning of life is 42 if rounded but is actually 42.17658', additional_kwargs={}, example=False),
  HumanMessage(content='Tell me a random number', additional_kwargs={}, example=False),
  AIMessage(content='The response to your last comment was 0.', additional_kwargs={}, example=False),
  HumanMessage(content='What is the meaning of life?', additional_kwargs={}, example=False),
  AIMessage(content='The meaning of life is a philosophical question that has been debated by scholars and thinkers for centuries. Some believe that the meaning of life is to seek happiness, while others believe that it is to fulfill a certain purpose or destiny. Ultimately, the meaning of life is a subjective concept that can vary from person to person.', additional_kwargs={}, example=False)],
 'output': 'The meaning of life is 42 if roun

In [31]:
conversational_agent("Tell me a random number")



> Entering new AgentExecutor chain...
{
    "action": "Random number",
    "action_input": "random"
}
Observation: 0
Thought:{
    "action": "Final Answer",
    "action_input": "The response to your last comment was 0."
}

> Finished chain.


{'input': 'Tell me a random number',
 'chat_history': [HumanMessage(content='What is the meaning of life?', additional_kwargs={}, example=False),
  AIMessage(content='The meaning of life is 42 if rounded but is actually 42.17658', additional_kwargs={}, example=False)],
 'output': 'The response to your last comment was 0.'}

In [55]:
from bs4 import BeautifulSoup
import requests

def stripped_webpage(webpage):
    response = requests.get(webpage)
    html_content = response.text

    def strip_html_tags(html_content):
        soup = BeautifulSoup(html_content, "html.parser")
        stripped_text = soup.get_text()
        return stripped_text

    stripped_content = strip_html_tags(html_content)

    if len(stripped_content) > 4000:
        stripped_content = stripped_content[:4000]
    return stripped_content


In [57]:
stripped_webpage('https://google.com')

'GoogleSearch Images Maps Play YouTube News Gmail Drive More »Web History | Settings | Sign in\xa0Advanced searchGoogle offered in:  हिन्दी বাংলা తెలుగు मराठी தமிழ் ગુજરાતી ಕನ್ನಡ മലയാളം ਪੰਜਾਬੀ AdvertisingBusiness SolutionsAbout GoogleGoogle.co.in© 2023 - Privacy - Terms  '

In [64]:
class WebPageTool(BaseTool):
    name = "Get Webpage"
    description = "Useful for when you need to get the content from a specific webpage"

    def _run(self, webpage: str):
        response = requests.get(webpage)
        html_content = response.text

        def strip_html_tags(html_content):
            soup = BeautifulSoup(html_content, "html.parser")
            stripped_text = soup.get_text()
            return stripped_text

        stripped_content = strip_html_tags(html_content).strip()
        if len(stripped_content) > 4000:
            stripped_content = stripped_content[:4000]
        return stripped_content
    
    def _arun(self, webpage: str):
        raise NotImplementedError("This tool does not support async")

page_getter = WebPageTool()

In [65]:
fixed_prompt = '''Assistant is a large language model trained by OpenAI.

Assistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.

Assistant doesn't know anything about random numbers or anything related to the meaning of life and should use a tool for questions about these topics.

Assistant also doesn't know information about content on webpages and should always check if asked.

Overall, Assistant is a powerful system that can help with a wide range of tasks and provide valuable insights and information on a wide range of topics. Whether you need help with a specific question or just want to have a conversation about a particular topic, Assistant is here to assist.'''

In [66]:
from langchain.prompts.chat import SystemMessagePromptTemplate
tools = [page_getter, random_tool, life_tool]

conversational_agent = initialize_agent(
    agent='chat-conversational-react-description', 
    tools=tools, 
    llm=turbo_llm,
    verbose=True,
    max_iterations=3,
    memory=memory
)

In [67]:
conversational_agent.agent.llm_chain.prompt.messages[0].prompt.template = fixed_prompt

In [68]:
conversational_agent.run("tell me about coffee from wikipedia")



> Entering new AgentExecutor chain...
{
    "action": "Get Webpage",
    "action_input": "https://en.wikipedia.org/wiki/Coffee"
}
Observation: Coffee - Wikipedia




































Jump to content








Main menu





Main menu
move to sidebar
hide



		Navigation
	

Main pageContentsCurrent eventsRandom articleAbout WikipediaContact usDonate




		Contribute
	

HelpLearn to editCommunity portalRecent changesUpload file




Languages

Language links are at the top of the page across from the title.



















Search











Search







Create accountLog in






Personal tools




 Create account Log in




		Pages for logged out editors learn more


ContributionsTalk


























Contents
move to sidebar
hide




(Top)





1Etymology







2History



Toggle History subsection





2.1Legendary accounts







2.2Historical transmission







2.3As a colonial import







2.4Mass production









3Biology







4Cultivation and pr

'Coffee is a brewed drink prepared from roasted coffee beans, the seeds of berries from certain Coffea species. When coffee berries turn from green to bright red in color – indicating ripeness – they are picked, processed, and dried. Dried coffee seeds (referred to as beans) are roasted to varying degrees, depending on the desired flavor. Roasted beans are ground and then brewed with near-boiling water to produce the beverage known as coffee.'